In [4]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [5]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../www/data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [6]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,GRATUIDAD
1,NUEVO BENEFICIARIO
2,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
3,EGRESO
4,BENEFICIARIO RENOVANTE
5,DESERCION


# Big Table with Summary

In [7]:
query_summary = """

SELECT id_2023 as id,
año_licitacion as año_solicitud,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado,
        ROUND(AVG(porcentaje_financiado),1) as porcentaje_financiado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_solicitud
"""

In [8]:
final_table_to_query = executes_query_duckdb(query_summary, cae)

In [9]:
final_table_to_query

,id,año_solicitud,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado,porcentaje_financiado
0,213.211,2011,M,Particular subvencionado,REGION DE LA ARAUCANIA,quintil 1 y 2,1,1,6,1,0,14180347.0,0.4
1,353.373,2012,F,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,2,4,1,0,5701697.0,0.3
2,117.359,2011,M,Municipal,REGION DEL MAULE,quintil 1 y 2,1,1,3,1,0,5402795.0,1.0
3,440.308,2010,F,Municipal,REGION METROPOLITANA,quintil 3,1,1,2,1,0,7515671.0,0.1
4,310.153,2010,F,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,3,6,1,0,15586471.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,1.216.867,2024,F,Particular Subvencionado,REGION METROPOLITANA,quintil 5,1,1,1,0,0,6083994.0,1.0
1290989,1.208.373,2024,F,Corporación Municipal,REGION DE COQUIMBO,quintil 1 y 2,1,1,1,0,0,3108000.0,1.0
1290990,1.230.560,2024,M,Particular Subvencionado,REGION DEL BIO-BIO,quintil 1 y 2,1,1,1,0,0,9426836.0,1.0
1290991,1.190.795,2024,F,Municipal DAEM,REGION DE LOS LAGOS,quintil 1 y 2,1,1,1,0,0,5043560.0,0.9


---

---

# Aggregated Total Borrowed

In [11]:
DOLAR = 950 # 1 dollar is $950 CLP

query_total_borrowed = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    round(SUM(total_prestado_ajustado)/{DOLAR}, 0) as total_prestado
FROM cae
GROUP BY region, quintil, genero, año_financiado
"""

total_borrowed = executes_query_duckdb(query_total_borrowed, cae)
# total_borrowed.to_json("../data/total_borrowed.json",
#                        orient="records",
#                        lines=False,
#                        force_ascii=False,
#                        indent=4)

total_borrowed

,region,quintil,genero,año_financiado,total_prestado
0,REGION DEL BIO-BIO,quintil 1 y 2,M,2011,24881777.0
1,REGION DE LOS RIOS,quintil 1 y 2,M,2011,3347841.0
2,REGION DE LOS LAGOS,quintil 1 y 2,M,2011,6005736.0
3,REGION DE ÑUBLE,quintil 1 y 2,M,2011,2988986.0
4,REGION DE LOS LAGOS,quintil 4,M,2011,2066734.0
...,...,...,...,...,...
2427,REGION DE ARICA Y PARINACOTA,quintil 5,M,2009,30496.0
2428,REGION DE MAGALLANES Y LA ANTARTICA,quintil 3,M,2007,44834.0
2429,REGION DE TARAPACA,quintil 5,M,2007,19861.0
2430,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2006,11411.0


---

---

# **Total CAES Granted**

In [12]:
query_total_granted = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    COUNT(tipo_beneficiario) as prestamos_entregados
FROM cae
WHERE tipo_beneficiario = 'NUEVO BENEFICIARIO'
GROUP BY region, quintil, genero, año_financiado
"""

total_granted = executes_query_duckdb(query_total_granted, cae)
total_granted

# total_granted.to_json("../data/total_cae_granted.json",
#                         orient="records",
#                         lines=False,
#                         force_ascii=False,
#                         indent=4)

,region,quintil,genero,año_financiado,prestamos_entregados
0,REGION METROPOLITANA,quintil 3,M,2011,3996
1,REGION DE ATACAMA,quintil 1 y 2,F,2007,100
2,REGION DE VALPARAISO,quintil 1 y 2,F,2011,3727
3,REGION DEL MAULE,quintil 5,M,2011,82
4,REGION DE VALPARAISO,quintil 3,M,2011,824
...,...,...,...,...,...
2299,REGION DE ATACAMA,quintil 1 y 2,F,2006,20
2300,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 5,F,2022,23
2301,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2008,13
2302,REGION DE TARAPACA,quintil 3,F,2006,9


---

---

# **AVG LOAN**

In [13]:
DOLAR = 950 # 1 dollar is $950 CLP

query_avg_loan = f"""
WITH money_per_loan as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        ROUND(SUM(total_prestado_ajustado)/{DOLAR},0) as total_prestado,
    FROM cae
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(total_prestado),0) as prestamo_promedio,
    COUNT(total_prestado) as prestamos_entregados

FROM money_per_loan
GROUP BY region, quintil, genero, año_entrega
"""

avg_loan = executes_query_duckdb(query_avg_loan, cae)

# avg_loan.to_json("../data/avg_loan.json",
#                          orient="records",
#                          lines=False,
#                          force_ascii=False,
#                          indent=4)

avg_loan

,region,quintil,genero,año_entrega,prestamo_promedio,prestamos_entregados
0,REGION DEL BIO-BIO,quintil 1 y 2,F,2014,7865.0,4203
1,REGION METROPOLITANA,quintil 5,F,2014,14569.0,3345
2,REGION DE VALPARAISO,quintil 1 y 2,F,2012,9899.0,3421
3,REGION DE COQUIMBO,quintil 4,M,2013,8810.0,188
4,REGION DE VALPARAISO,quintil 4,F,2013,11727.0,966
...,...,...,...,...,...,...
2356,REGION DE VALPARAISO,quintil 5,M,2008,5838.0,2
2357,REGION DE LOS RIOS,quintil 5,F,2008,20498.0,1
2358,REGION DE COQUIMBO,quintil 5,M,2010,31746.0,1
2359,REGION DE ANTOFAGASTA,quintil 5,F,2010,12774.0,1


---

---

# **FINANCED YEARS**

In [20]:
query_financed_years = f"""
WITH prov_years_financed as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        COUNT(total_prestado_ajustado) as años_financiados
    FROM cae
    WHERE total_prestado_ajustado > 0
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(años_financiados),2) as años_financiados,
    COUNT(años_financiados) as prestamos_entregados

FROM prov_years_financed
GROUP BY region, quintil, genero, año_entrega
"""

financed_years = executes_query_duckdb(query_financed_years, cae)

financed_years.to_json("../www/data/financed_years.json",
                          orient="records",
                          lines=False,
                          force_ascii=False,
                          indent=4)

financed_years

,region,quintil,genero,año_entrega,años_financiados,prestamos_entregados
0,REGION DE ANTOFAGASTA,quintil 4,F,2014,3.83,195
1,REGION DEL BIO-BIO,quintil 4,M,2008,4.22,406
2,REGION DE LOS RIOS,quintil 1 y 2,F,2013,3.86,687
3,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,F,2010,4.37,41
4,REGION METROPOLITANA,quintil 3,M,2012,4.01,3668
...,...,...,...,...,...,...
2356,REGION DE LOS LAGOS,quintil 5,F,2008,4.50,2
2357,REGION DE ANTOFAGASTA,quintil 5,F,2010,5.00,1
2358,REGION DEL BIO-BIO,quintil 5,M,2010,5.50,2
2359,REGION DEL MAULE,quintil 5,F,2010,4.00,1


---

---

In [19]:
h = executes_query_duckdb("select * from cae where total_prestado_ajustado > 0", cae)
h

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,porcentaje_financiado,total_prestado_ajustado
0,667.946,2012,2012,F,Chilena,REGION DE VALPARAISO,quintil 4,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,2043680,2043680,Profesional con Licenciatura,Diurna,ESTADO,1.000000,3449731.840
1,958.754,2017,2017,F,Chilena,REGION DE LA ARAUCANIA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,7,1,Particular subvencionado,3674581,3674582,Profesional con Licenciatura,Diurna,ESTADO,1.000000,5250976.249
2,668.482,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Particular subvencionado,820429,2109075,Profesional con Licenciatura,Diurna,ESTADO,0.388999,1384884.152
3,676.859,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,3,1,Municipal,1113000,1113000,Técnico,Diurna,ESTADO,1.000000,1878744.000
4,677.231,2012,2012,F,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,1523626,1523626,Profesional,Diurna,ESTADO,1.000000,2571880.688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254771,700.787,2018,2024,M,Chilena,REGION METROPOLITANA,quintil 4,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,4,4,Municipal,1460000,2714368,Profesional,Vespertina,SCOTIABANK,0.537878,2920000.000
4254772,1.168.744,2023,2024,M,Chilena,REGION DE COQUIMBO,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,5,2,Municipal DAEM,2589000,2994134,Profesional con Licenciatura,Diurna,ESTADO,0.864691,5178000.000
4254773,1.132.655,2022,2024,F,Chilena,REGION METROPOLITANA,quintil 5,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,5,3,Particular subvencionado,3070924,3070924,Profesional con Licenciatura,Diurna,ESTADO,1.000000,6141848.000
4254774,1.156.179,2023,2024,F,Chilena,REGION DE COQUIMBO,quintil 5,NO,BENEFICIARIO RENOVANTE,CURSO SUPERIOR,...,7,2,Particular Pagado,4855324,4855324,Profesional con Licenciatura,Diurna,ESTADO,1.000000,9710648.000


In [ ]:
print(len(h["region_grupo_familiar"].unique()), h["region_grupo_familiar"].unique())

16 ['REGION DE VALPARAISO' 'REGION DE LA ARAUCANIA' 'REGION METROPOLITANA'
 'REGION DE LOS LAGOS' "REGION DEL LIB. B. O'HIGGINS" 'REGION DEL BIO-BIO'
 'REGION DE LOS RIOS' 'REGION DE ANTOFAGASTA' 'REGION DE TARAPACA'
 'REGION DEL MAULE' 'REGION DE COQUIMBO' 'REGION DE ARICA Y PARINACOTA'
 'REGION DE AYSEN DEL G. C. IBAÑEZ' 'REGION DE MAGALLANES Y LA ANTARTICA'
 'REGION DE ATACAMA' 'REGION DE ÑUBLE']


In [ ]:
final_table_to_query.to_parquet("../data/summary_table.parquet")